In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import torch
from modelV2.tune import HyperParamOptimizer
from modelV2.train import tuning_wrapper
# from modelV2.arch.rn_2 import ResNet18
from torchvision.transforms import v2
import os
import re
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

/home/tbrow51/.conda/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_masked_path(target_mask_factor: str, mask_factors_str: str, masked_paths_str: str):
    try:
        # parse the mask factors/masked path strings
        mask_factors_list = parse_mask_factors(mask_factors_str)
        mask_paths_list = parse_masked_paths(masked_paths_str)

        # find the index of the target mask factor
        target_idx = mask_factors_list.index(target_mask_factor)

        # extract the target masked path
        return mask_paths_list[target_idx]
    
    except Exception as e:
        print(e)
        return 'CAUGHT_ERROR'

def parse_mask_factors(mask_factors_str: str):
    # remove brackets and spaces from the string
    mask_factors_str = mask_factors_str.translate({ord(c): None for c in "][ "})
    # the split on commas
    return mask_factors_str.split(',')

def parse_masked_paths(masked_paths_str: str):
    # split on dollar characters (pls dont have any of these in your paths!!)
    return masked_paths_str.split('$')

In [3]:
SEED = 49713
BASE_DIR = "/mnt/NAS3/DataBalance/balance-study/data/images/run_e1_a_1200_800/"
MASK_STR = "0_0"
BALANCE_DISTRIBUTIONS = True

N_BOOTSTRAPS = 2
EPOCHS_PER_RUN = 1
MONITOR_METRIC = 'loss'
HPARAM_RANGE_DICT = {
    'learning_rate': (0.00001, 0.01),
}


BATCH_SIZE = 48
N_RANDOM = 5
N_GUIDED = 10
MODEL_NAME = f"rn18_no_bal_a"

In [4]:
pos_df = pd.read_csv(os.path.join(BASE_DIR, "pos_out_df.csv"))
neg_df = pd.read_csv(os.path.join(BASE_DIR, "neg_proc_df.csv"))

# only include exams we've assigned to the tuning set
# this is done so evaluation class balance is consistent 
# between train/val/test sets
# 10/90 balance testing can be done after tuning is completed
neg_df = neg_df[neg_df["include_in_tuning"] == 1]

pos_df.dropna(subset=['masked_factors', 'masked_png_paths'], inplace=True)
neg_df.dropna(subset=['masked_factors', 'masked_png_paths'], inplace=True)
print(pos_df[['assigned_split']].value_counts())
print(neg_df[['assigned_split']].value_counts())

def correct_paths_to_kraken(hiti_path):
    replace_str = r"/mnt/NAS3/DataBalance/balance-study\1"
    # use re.sub to replace everything before '/data/images'
    return re.sub(r'^.*?(/data/images)', replace_str, hiti_path)

pos_df['__target_path'] = pos_df.apply(
    lambda x: correct_paths_to_kraken(extract_masked_path(
        MASK_STR.replace('_', '.'), 
        x.masked_factors, 
        x.masked_png_paths
    )), 
    axis=1
)

neg_df['__target_path'] = neg_df.apply(
    lambda x: correct_paths_to_kraken(extract_masked_path(
        MASK_STR.replace('_', '.'), 
        x.masked_factors, 
        x.masked_png_paths
    )), 
    axis=1
)

DF_POOL_DICT = {
    "pos": {
        "train": pos_df[pos_df.assigned_split == "train"],
        "val": pos_df[pos_df.assigned_split == "val"],
        "test": pos_df[pos_df.assigned_split == "test"]
    },
    "neg": {
        "train": neg_df[neg_df.assigned_split == "train"],
        "val": neg_df[neg_df.assigned_split == "val"],
        "test": neg_df[neg_df.assigned_split == "test"]
    }
}

assigned_split
train             4485
test               986
val                955
Name: count, dtype: int64
assigned_split
train             45232
val                9743
test                991
Name: count, dtype: int64


In [5]:
AUGMENT_DICT = {
    'crop': {
        'enabled': True, 
        'func': v2.RandomResizedCrop,
        'prob': 1.0,
        'params': {
            'size': (1200, 800), 
            'scale': (0.7, 1.0),
        }
    },
    'rotation': { 
        'enabled': True, 
        'func': v2.RandomRotation,
        'prob': 0.4,
        'params': {
            'degrees': 5
        }
    },
    'color_jitter': {
        'enabled': True,
        'func': v2.ColorJitter,
        'prob': 0.2,
        'params': {
            'brightness': 0.4, 
            'contrast': 0.4,
            'saturation': 0.4, 
            'hue': 0.2
        }
    },
    'gaussian_blur': {
        'enabled': True,
        'func': v2.GaussianBlur,
        'prob': 0.2,
        'params': {
            'kernel_size': 3
        }
    }
}

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device...")

using cuda device...


In [7]:
# get optimizer
hparam_optimizer = HyperParamOptimizer(
    hparam_range_dict=HPARAM_RANGE_DICT, 
    balance=BALANCE_DISTRIBUTIONS,
    monitor_metric=MONITOR_METRIC,
    n_bootstraps=N_BOOTSTRAPS,
    epochs_per_run=EPOCHS_PER_RUN,
    seed=SEED
)

# set device and model
hparam_optimizer.set_model(
    device=device, 
    model_type='resnet18'
)

# load data into optimizer
hparam_optimizer.load_data(
    df_pool_dict=DF_POOL_DICT,
    batch_size=BATCH_SIZE,
    augment_dict=AUGMENT_DICT
)

# start optimizer
hparam_optimizer.optimize(
    objective=tuning_wrapper,
    n_random=N_RANDOM, 
    n_guided=N_GUIDED, 
    model_name=MODEL_NAME
)


Data loaded...

Optimizing model ------------------------------------------------------------
Logging results to './logs/rn18_no_bal_a/opt_log.json'
|   iter    |  target   | learni... |
-------------------------------------
running 2 bootstraps...
with seeds: [4266198 9960322]
reference dataframe:
Patients: 2,044
Exams: 2,067
Images: 3,139

target dataframe:
Patients: 24,189
Exams: 38,648
Images: 45,232

balancing distribution based on tissueden, ViewPosition, Race

balancing data with 1.00 : 1.00 ratio

reference dataframe:
Patients: 425
Exams: 427
Images: 668

target dataframe:
Patients: 5,184
Exams: 8,293
Images: 9,743

balancing distribution based on tissueden, ViewPosition, Race

balancing data with 1.00 : 1.00 ratio



/home/tbrow51/.conda/envs/torch/lib/python3.9/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")



Epoch 0 ----------------------------------------


 45%|████▌     | 59/131 [00:39<00:48,  1.49batch/s, train_loss=0.692, train_acc=0.526, train_auc=0.557, train_f1=0.656, train_prec=0.514, train_rec=0.904]


KeyboardInterrupt: 

In [ ]:
# opt = pd.read_json("/mnt/NAS3/DataBalance/balance-study/mammo-balance-study/logs/rn18_no_bal_a/opt_log.json", lines=True)
# opt
# # demo

In [ ]:
# demo = pd.read_json("/mnt/NAS3/DataBalance/balance-study/mammo-balance-study/logs/rn18_no_bal_a/demographics.json", lines=True)
# demo